In [1]:
import cv2 as cv
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pickle
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from numpy.linalg import norm
from collections import defaultdict

In [ ]:
##Main Faceloading
#urutan pertama untuk menentukan diretory yang mau digunakanb
class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (150, 150)
        self.x = []       # list images
        self.y = []       # list labels
        self.paths = []   # list file paths ⬅️ tambahan
        self.detector = MTCNN()


    def extract_faces(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # turunkan resolusi biar cepat
        max_size = 640
        h, w = img.shape[:2]
        scale = max(h, w) / max_size
        if scale > 1:
            img = cv.resize(img, (int(w/scale), int(h/scale)))

        results = self.detector.detect_faces(img)
        if len(results) == 0:
            raise ValueError("No face detected")

        x, y, w, h = results[0]['box']
        x, y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        self.target_size = (150, 150)
        face_array = cv.resize(face, self.target_size)
        return face_array

    def load_faces(self, dir):
        FACES, FILES = [], []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)
                single_face = self.extract_faces(path)
                FACES.append(single_face)
                FILES.append(path)  # simpan path
            except Exception as e:
                print(f"Error processing {im_name}: {e}")
        return FACES, FILES

    def load_classes(self):
        has_classes = False
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if os.path.isdir(path):
                has_classes = True
                faces, files = self.load_faces(path)
                labels = [sub_dir for _ in range(len(faces))]
                print(f"[{sub_dir}] loaded successfully: {len(labels)} images")
                self.x.extend(faces)
                self.y.extend(labels)
                self.paths.extend(files)  # simpan path juga

        if not has_classes:
            print("No folders found. Assuming flat structure.")
            faces, files = self.load_faces(self.directory + '/')
            labels = ['unknown' for _ in range(len(faces))]
            print(f"[unknown] loaded successfully: {len(labels)} images")
            self.x.extend(faces)
            self.y.extend(labels)
            self.paths.extend(files)

        return np.asarray(self.x), np.asarray(self.y), np.asarray(self.paths)


In [ ]:
# faceloading = FACELOADING("/content/drive/MyDrive/dataset")
faceloading = FACELOADING("Dataset")
x, y, paths = faceloading.load_classes()

Error processing IMG_4938.HEIC: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing IMG_4970.HEIC: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

[Alejandra] loaded successfully: 15 images
Error processing IMG_4939.HEIC: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing IMG_4971.HEIC: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

[Alessandro] loaded successfully: 15 images
Error processing IMG_4940.HEIC: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Ass

In [5]:
embeder = FaceNet()
def get_embedding(face_img):
  face_img = face_img.astype('float32')
  face_img = np.expand_dims(face_img, axis=0)
  embedding = embeder.embeddings(face_img)
  return embedding[0]

In [6]:
EMBEDDED_X = []
LABELS_Y = []
SPOOF_LABELS = []
SPOOF_FILES = []

for face_img, label, path in zip(x, y, paths):
  embedding = get_embedding(face_img)
  EMBEDDED_X.append(embedding)
  LABELS_Y.append(label)

np.savez_compressed('faces_embeddings_done_4classes.npz', EMBEDDED_X, LABELS_Y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
print(MTCNN.__version__)

AttributeError: module 'mtcnn' has no attribute '__version__'